In [1]:
!git clone https://github.com/japolak/datahow-course-scripts.git
!pip install --upgrade scipy==1.7.3
!pip install --upgrade pyDOE2
!cd /content/datahow-course-scripts

Cloning into 'datahow-course-scripts'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 89 (delta 34), reused 69 (delta 17), pack-reused 0
Unpacking objects: 100% (89/89), done.
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-py3-none-any.whl size=25539 sha256=ff9b8f85d81d4da5d081165a681c5eb8f875e62841de16237d5a5f8b5697bc0d
  Stored in directory: /root/.cache/pip/wheels/49/91/2d/d08e80806bf7756193541f6c03c0492a

In [1]:
import pandas as pd
import numpy as np
import scipy
import pyDOE2
import importlib  
import scipy.integrate



# import custom modules
transformations = importlib.import_module("datahow-course-scripts.modules.transformations")
emulator = importlib.import_module("datahow-course-scripts.modules.emulator")

#notebook settings
pd.set_option('display.max_rows', 30)


#owu = transformations.owu_from_csv("/content/mytable.csv")




ModuleNotFoundError: No module named 'datahow-course-scripts'

# Experiment run generation

In [5]:
''' One Experiment run definition (variable = value)'''
# Model parameters
mu_g_max = 0.05;
mu_d_max = 0.025;
K_g_Glc  = 1;
K_I_Lac  = 30;
K_d_Lac  = 50;
k_Glc    = 0.04;
k_Lac    = 0.06;
k_Prod   = 1;
model_param = [mu_g_max,mu_d_max,K_g_Glc,K_I_Lac,K_d_Lac,k_Glc,k_Lac,k_Prod]

# Process parameters
feed_start = 3.0
feed_end = 12.0
Glc_feed_rate = 0.50
Glc_0 = 12.0
VCD_0 = 0.5
process_param = [feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0]


In [6]:
# Experiment run generation
t,y = emulator.predict_chrom_phase(model_param,process_param)
run_df = pd.DataFrame(y,columns=["X:VCD", "X:Glc", "X:Lac", "X:Titer"])
run_df


,X:VCD,X:Glc,X:Lac,X:Titer
0,0.500000,12.000000,0.000000,0.000000
1,1.471551,11.134896,1.303268,0.199102
2,3.869848,8.732356,4.925306,1.952155
3,7.907237,3.195349,13.306272,16.391986
4,12.101564,5.673504,27.739503,67.774204
5,14.782388,4.689308,47.397539,175.129793
6,14.749745,2.504735,68.981167,325.323861
7,12.664054,1.598027,88.865400,469.944302
8,10.290136,2.880981,105.346429,599.963298
9,8.330067,6.080447,118.708868,723.413889


# DOE Dataset Generation

In [9]:
''' DOE Dataset definition (variable = [lower bound, upper bound]) '''
# Model parameters
mu_g_max = [0.01, 0.10]
mu_d_max = [0.025, 0.075]
K_g_Glc = [1, 2]
K_g_Lac = [50, 80]
K_I_Lac = [30, 50]
K_d_Lac = [50, 80]
k_Glc = [0.04, 0.12]
k_Lac = [0.06, 0.15]
k_Prod = [1, 3]

# Process parameters
feed_start = [3, 6]
feed_end = [12.0, 16]
Glc_feed_rate = [0.50, 2.0]
Glc_0 = [12.0, 15.0]
VCD_0 = [0.3, 0.6]

var_lims = {"mu_g_max":mu_g_max,
    "mu_d_max": mu_d_max,
    "K_g_Glc" : K_g_Glc,
    "K_g_Lac" : K_g_Lac,
    "K_I_Lac" : K_I_Lac,
    "K_d_Lac" : K_d_Lac,
    "k_Glc" : k_Glc,
    "k_Lac" : k_Lac,
    "k_Prod" : k_Prod,
    "feed_start" : feed_start,
    "feed_end" : feed_end,
    "Glc_feed_rate" : Glc_feed_rate,
    "Glc_0" : Glc_0,
    "VCD_0" : VCD_0}


In [13]:
# DOE Dataset generation
num_runs = 5
filename = "mytable.csv"
owu_df = emulator.generate_data(var_lims, num_runs, filename)
owu_df

,timestamps,X:VCD,X:Glc,X:Lac,X:Titer
0,0.0,0.450000,1.350000e+01,0.000000,0.000000
1,1.0,1.408693,1.187831e+01,2.136868,0.613499
2,2.0,3.628148,7.285377e+00,8.197172,6.236986
3,3.0,5.094934,6.583614e-08,20.251589,33.572718
4,4.0,3.617572,-1.257750e-09,31.217884,33.572718
5,5.0,3.042765,1.222307e+01,39.073077,52.370665
6,6.0,3.409265,3.600729e+01,47.249572,96.388295
7,7.0,3.371678,5.940844e+01,55.919840,149.603029
8,8.0,2.496128,8.362869e+01,63.511171,201.790335
9,9.0,1.308790,1.100347e+02,68.230753,236.695544


In [15]:
owu = transformations.owu_from_csv("/content/mytable.csv")
owu

X:VCD         X:Glc       X:Lac      X:Titer
run timestamps                                                 
 0  0.0         0.450000  1.350000e+01    0.000000     0.000000
    1.0         1.408693  1.187831e+01    2.136868     0.613499
    2.0         3.628148  7.285377e+00    8.197172     6.236986
    3.0         5.094934  6.583614e-08   20.251589    33.572718
    4.0         3.617572 -1.257750e-09   31.217884    33.572718
    5.0         3.042765  1.222307e+01   39.073077    52.370665
    6.0         3.409265  3.600729e+01   47.249572    96.388295
    7.0         3.371678  5.940844e+01   55.919840   149.603029
    8.0         2.496128  8.362869e+01   63.511171   201.790335
    9.0         1.308790  1.100347e+02   68.230753   236.695544
    10.0        0.613295  1.382673e+02   70.551438   254.512960
    11.0        0.280147  1.674503e+02   71.624147   262.889778
    12.0        0.126824  1.970787e+02   72.111929   266.727776
    13.0        0.057207  2.269108e+02   72.332323   268.467743
    14.0        0.025767  2.568352e+02   72.431659   269.253134
 1  0.0         0.562500  1.387500e+01    0.000000     0.000000
    1.0         2.260590  1.239156e+01    2.792046     1.061105
    2.0         6.357994  7.444527e+00   12.115506    19.290121
    3.0         7.581423 -7.462181e-08   29.779970   100.129970
    4.0         4.083008  3.800762e-09   42.644181   100.129970
    5.0         2.005914 -1.592649e-09   49.251386   100.129970
    6.0         1.097739  5.686625e+00   52.489618   110.001986
    7.0         0.877247  2.101435e+01   54.697252   135.382127
    8.0         0.697164  3.657358e+01   56.464373   156.132180
    9.0         0.530061  5.233840e+01   57.844423   172.632150
    10.0        0.353311  6.830688e+01   58.841859   184.722511
    11.0        0.177691  8.449209e+01   59.432474   191.946917
    12.0        0.068985  1.008523e+02   59.694734   195.170297
    13.0        0.024324  1.090517e+02   59.791390   196.360608
    14.0        0.008510  1.090336e+02   59.825275   196.778206
 2  0.0         0.412500  1.237500e+01    0.000000     0.000000
    1.0         1.038088  1.123616e+01    1.164093     0.349157
    2.0         2.403707  8.489610e+00    3.973950     2.178753
    3.0         4.547128  2.745500e+00    9.878091    13.936779
    4.0         4.358815  4.240178e-09   17.932460    25.561003
    5.0         5.680196  3.095100e+01   26.155269    53.373728
    6.0         7.138896  6.352931e+01   37.284032   105.614216
    7.0         6.227944  9.539228e+01   49.136470   180.713978
    8.0         3.979551  1.303433e+02   57.842005   247.540700
    9.0         2.283095  1.686868e+02   63.092406   292.168851
    10.0        1.246864  2.093014e+02   66.030152   318.466230
    11.0        0.663909  2.512455e+02   67.614196   333.025382
    12.0        0.348916  2.939224e+02   68.452144   340.831967
    13.0        0.182147  3.369912e+02   68.891040   344.949397
    14.0        0.094761  3.585186e+02   69.119767   347.102860
    0.0         0.487500  1.312500e+01    0.000000     0.000000
-1  1.0         0.743331  1.152351e+01    1.699363     0.346443
    2.0         1.063618  9.160058e+00    4.209228     1.296277
    3.0         1.390074  5.933219e+00    7.642360     3.775630
    4.0         1.585586  2.006811e+00   11.849146     9.714413
    5.0         1.398503  2.757551e+00   16.076639    14.453775
    6.0         1.477254  2.448907e+01   20.078962    19.805416
    7.0         1.505565  4.603532e+01   24.263471    25.139748
    8.0         1.384509  6.767714e+01   28.344510    31.182062
    9.0         1.023504  8.995668e+01   31.750143    36.957331
    10.0        0.628209  1.133003e+02   34.030127    41.202079
    11.0        0.361717  1.375227e+02   35.380864    43.857296
    12.0        0.203508  1.622959e+02   36.149292    45.413036
    13.0        0.113209  1.873892e+02   36.579191    46.297029
    14.0        0.062599  2.126637e+02   36.817618    46.791251
 4  0.0         0.337500  1.462500e+01    0.000000 

# Create a Design of Experiments

In this section, we first create a set of experiments using a Latin Hypercube design (LHD). This will be used to create an observation-wise unfolder (OWU) matrix that will be analyzed using Principle Component Analysis (PCA) and also in later scripts.

*Note: the created experimental design will be saved in the file "Experiments_TrainSet.mat" and used for comparison in the following lecture scripts.* 

For creating a DOE file, the following needs to be specified:
*   Number of experiments
*   Design limits
*   Errors 




In [ ]:
## Define the number of experiments
Nruns = 24

In [ ]:
## Define design limits
glc_start_range = [10,80]
vcd_start_range = [0.1,1]
feed_start_range = [1,4]
feed_end_range = [8,12]
feed_rate_range = [5,20]

In [ ]:
## Define errors
inputs_std_error = [0,0]  # absolute error on [VCD, Glc]
outputs_std_error = [0,0,0,0,0] # absolute error on [VCD, Glc, Lac, Titer, Aggr]

## Create design

In [ ]:
# TODO: 
# lhsdesign function
# add path
# save file
# load file

## Plot experiments

In [ ]:
# TODO:
# plot VCD profiles
# plot Glucose profiles
# plot Lactate profiles
# plot Titer profiles